In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
#import joblib
import os
import pickle
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from itertools import product
import warnings
import gc

%matplotlib inline
plt.rcParams["figure.figsize"] = [16,7]

/home/dmitry/miniconda3/envs/tf_intel/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%%time

#test_csv = pd.read_csv('../input/test.csv')
train_csv = pd.read_csv(
    '../input/train.csv',
    dtype={
        'id': np.int64,
        'store_nbr': np.int64,
        'item_nbr': np.int64,
        'onpromotion': np.bool
    },
    converters={
        'unit_sales': lambda x: float(x) if float(x) > 0 else 0,
    },
    parse_dates=[1],
    skiprows=range(1, 38594265) # 2015-01-01
)

CPU times: user 3min 13s, sys: 13.3 s, total: 3min 26s
Wall time: 3min 15s


#### Get the set of items

In [3]:
train_csv.date.max() - train_csv.date.min()

Timedelta('957 days 00:00:00')

In [27]:
u_dates = pd.date_range(train_csv.date.min(), train_csv.date.max())
u_stores = train_csv.store_nbr.unique()
u_items = train_csv.item_nbr.unique()

df_train = train_csv.set_index(["date", "store_nbr", "item_nbr"])
df_train = df_train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)

In [28]:
df_train.drop(['id'], inplace=True, axis=1)
df_train.loc[:, "unit_sales"].fillna(0, inplace=True)
df_train.loc[:, "onpromotion"].fillna(False, inplace=True)
df_train["onpromotion"] = np.int64(df_train["onpromotion"])
df_train.reset_index(inplace=True)

#df_train.xs((24, 2058758), level=(1,2))

In [29]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2015-01-01,25,103665,12.0,0
1,2015-01-01,25,105575,23.0,0
2,2015-01-01,25,108634,1.0,0
3,2015-01-01,25,108698,6.0,0
4,2015-01-01,25,108786,6.0,0


In [30]:
df_train.shape

(208790352, 5)

In [19]:
test_csv = pd.read_csv(
    '../input/test.csv',
    parse_dates=[1],
    dtype={
        'item_nbr': np.int32,
        'store_nbr': np.int8,
        'unit_sales': np.float32,
    },
    converters={
        'onpromotion': lambda x: 1 if x == 'True' else 0
    }
)
df_test['unit_sales'] = 0

In [31]:
u_dates = pd.date_range(test_csv.date.min(), test_csv.date.max())
u_stores = train_csv.store_nbr.unique()
u_items = train_csv.item_nbr.unique()

df_test = test_csv.set_index(["date", "store_nbr", "item_nbr"])
df_test = df_test.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)
df_test.drop(['id'], inplace=True, axis=1)
df_test.reset_index(inplace=True)
df_test['unit_sales'] = 0

In [32]:
df_test.head()

,date,store_nbr,item_nbr,onpromotion,unit_sales
0,2017-08-16,25,103665,0.0,0
1,2017-08-16,25,105575,0.0,0
2,2017-08-16,25,108634,0.0,0
3,2017-08-16,25,108698,0.0,0
4,2017-08-16,25,108786,0.0,0


In [33]:
df_train = pd.concat([df_train, df_test]).iloc[:, [0,3,1,4,2]]
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2015-01-01,25,103665,12.0,0.0
1,2015-01-01,25,105575,23.0,0.0
2,2015-01-01,25,108634,1.0,0.0
3,2015-01-01,25,108698,6.0,0.0
4,2015-01-01,25,108786,6.0,0.0


In [34]:
df_train.tail()

,date,store_nbr,item_nbr,unit_sales,onpromotion
3487099,2017-08-31,52,2122818,0.0,0.0
3487100,2017-08-31,52,2011459,0.0,0.0
3487101,2017-08-31,52,2126944,0.0,0.0
3487102,2017-08-31,52,2123839,0.0,0.0
3487103,2017-08-31,52,2011451,0.0,0.0


In [35]:
items_attr = pd.read_csv(
    '../input/items.csv',
    dtype={
        'item_nbr': np.int32,
        'class': np.int8,
        'perishable': np.int8,
        
    }
)

In [36]:
df_holidays = pd.read_csv('../input/holidays_events.csv')
df_holidays = df_holidays[df_holidays['transferred'] == False]
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [38]:
df_stores = pd.read_csv('../input/stores.csv')
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [43]:
stores = []
df_store_holidays = None
for r in df_holidays.iterrows():
    if r[1]['locale'] == 'Local':
        affected_stores = df_stores[
            df_stores['city'] == r[1]['locale_name']
        ].copy()
    elif r[1]['locale'] == 'Regional':
        affected_stores = df_stores[
            df_stores['state'] == r[1]['locale_name']
        ].copy()
    else:
        affected_stores = df_stores.copy()
    affected_stores['date'] = r[1]['date']
    if df_store_holidays is None:
        
        df_store_holidays = affected_stores.copy()
    else:
        df_store_holidays = pd.concat([df_store_holidays, affected_stores])
    
df_store_holidays = df_store_holidays[['store_nbr', 'date']]
df_store_holidays['holiday'] = 1
df_store_holidays['date'] = pd.to_datetime(df_store_holidays['date'])
df_store_holidays = df_store_holidays.drop_duplicates()


In [44]:
#df_store_holidays = pd.read_csv('../store_holidays.csv')
df_store_holidays['date'] = pd.to_datetime(df_store_holidays['date'])

u_date = pd.date_range(df_train.date.min(), df_train.date.max()).unique()

u_store_nbr = df_store_holidays.store_nbr.unique()

idx_cols = ['date', 'store_nbr']
df_store_holidays.set_index(idx_cols, inplace=True)

mindex = pd.MultiIndex.from_product(
    (u_date, u_store_nbr),
    names=idx_cols
)
df_store_holidays = df_store_holidays.reindex(mindex)

df_store_holidays.fillna(0, inplace=True)
df_store_holidays['holiday'] = np.int8(df_store_holidays['holiday'])
df_store_holidays.reset_index(inplace=True)

In [45]:
df_w_hol_dow = df_train.merge(df_store_holidays)

In [46]:
df_w_hol_dow['dow'] = df_w_hol_dow.date.dt.dayofweek

In [47]:
df_w_hol_dow['weekend'] = df_w_hol_dow['dow'] // 5

In [48]:
%%time
df_w_hol_dow['waged_day'] = df_w_hol_dow['date'].apply(lambda x: 1 if x.day == 15 or x.day == x.daysinmonth else 0)

CPU times: user 1h 6min 56s, sys: 5min 45s, total: 1h 12min 42s
Wall time: 1h 10min 14s


In [49]:
df_train_reduced_cleared['onpromotion'] = np.int8(df_train_reduced_cleared['onpromotion'])
df_train_reduced_cleared = df_w_hol_dow

In [55]:
%%time
df_train_reduced_cleared.sort_values(
    ['store_nbr', 'item_nbr']
).to_csv(
    'data/df_reduced_cleared.csv.gz',
    compression='gzip'
)

CPU times: user 1h 20min 55s, sys: 18.1 s, total: 1h 21min 14s
Wall time: 1h 21min 2s


In [56]:
df_train_reduced_cleared.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,holiday,dow,weekend,waged_day
0,2015-01-01,25,103665,12.0,0,1,3,0,0
1,2015-01-01,25,105575,23.0,0,1,3,0,0
2,2015-01-01,25,108634,1.0,0,1,3,0,0
3,2015-01-01,25,108698,6.0,0,1,3,0,0
4,2015-01-01,25,108786,6.0,0,1,3,0,0


In [51]:
df_w_hol_dow.shape

(212277456, 9)

In [38]:
df_w_hol_dow.shape

(30966138, 8)

In [25]:
df_train_reduced_cleared.shape

(173368242, 8)

In [41]:
df_train_reduced_cleared.shape

(16142338, 8)

In [ ]:
%%time
df_w_hol_dow.sort_values(['store_nbr', 'item_nbr']).to_csv(
    '../input_cleared/df_w_hol_dow_w_xmas.csv.gz',
    compression='gzip'
)

In [30]:
#df_train.sort_values(['store_nbr', 'item_nbr']).to_csv('data/df_train_cleared.csv.gz', compression='gzip')